# Import libraries

In [166]:
# Import libraries
import rasterio
import numpy as np
import matplotlib
from matplotlib import pyplot
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Extract raster values

In [125]:
# Read raster file containing 4 GHS settlement classes
smod_world_4class = rasterio.open('../01_data/gis/world_4class_binary.tif')

# Extract raster profile
profile = smod_world_4class.profile.copy()

# Extract raster data as numpy array
smod_world_4class_data = smod_world_4class.read()

# Check extracted data
smod_world_4class_data

array([[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]]],
      dtype=float32)

In [79]:
# Create bool array aka mask
mask = np.isin(smod_world_4class_data.copy(), [-9999., 0], invert=False)

# set everything where mask is True to NoData
smod_world_4class_data[mask] = profile["nodata"]

# Save raster file
with rasterio.open('world_4class_1_ND.tif', "w", **profile) as output:
    output.write(smod_world_4class_data)

# Reproject raster to WGS

In [177]:
# Read raster file
smod_world_4class_1ND = rasterio.open('../01_data/gis/world_4class_1_ND.tif')

# Print source CRS
print('Source raster CRS:',smod_world_4class_1ND.crs)

Source raster CRS: ESRI:54009


In [179]:
# Source and target CRS
target_crs = {'init': 'EPSG:4326'}
src_crs = smod_world_4class_1ND.crs

# Calculate transform array and shape of reprojected raster
transform, width, height = calculate_default_transform(src_crs,
                                                       target_crs,
                                                      smod_world_4class_1ND.width,
                                                      smod_world_4class_1ND.height,
                                                      *smod_world_4class_1ND.bounds)

In [189]:
# Update the meta data for the target/transformed raster
kwargs = smod_world_4class_1ND.meta.copy()

kwargs.update(
    {
        'crs': target_crs,
        'transform': transform,
        'width': width,
        'height': height
    }
)

In [194]:
# Open target raster
target_raster = rasterio.open('../01_data/gis/world_4class_1_ND_WGS84.tif', 'w', **kwargs)

In [204]:
# Reproject and save raster data
for i in range(1, smod_world_4class_1ND.count+1):
    reproject(
        source=rasterio.band(smod_world_4class_1ND, i),
        destination=rasterio.band(target_raster, i),
        src_transform=smod_world_4class_1ND.transform,
        src_crs=smod_world_4class_1ND.crs,
        dst_transform=transform,
        dst_crs=target_crs,
        resampling=Resampling.nearest
    )

In [206]:
target_raster.close()